In [1]:
message_structure_keys = [('Message-ID', False), ('Date', True), ('From', True), ('To', True), ('Subject', True), 
                        ('Mime-Version', False), ('Content-Type',False),('Content-Transfer-Encoding',False),
                        ('X-From',False),('X-To',False), ('X-cc',False), ('X-bcc',False),
                        ('X-Folder',False), ('X-Origin',False), ('X-FileName',False)]

In [2]:
def extract_value(key, line):
    return line.split(key + ':')[1].strip()

In [3]:
def transform_message(keys, message):
    try:
        message_structure = {}
        body = ''
        for key in keys:
            for line in message:
                if key[0] in line:
                    if key[1]:
                        message_structure.update({key[0]:extract_value(key[0], line)})
                    message.remove(line)
                    break
        message_structure.update({'body':"".join(message).strip()})
    except:
        pass
    return message_structure

In [119]:
import pandas as pd
import numpy as np

In [58]:
# Enron Emails can be found here https://www.cs.cmu.edu/~./enron/
text = pd.read_csv("data/emails.csv.zip", chunksize=10)
temp = next(text)

In [59]:
temp = temp.drop(columns=["file"])

In [60]:
temp2 = temp.apply(lambda x : x.str.split('\n'), axis = 1)

In [61]:
temp3 = []
for index, rows in temp2.iterrows():
    temp3.append(transform_message(message_structure_keys, rows.message))

In [62]:
df = pd.DataFrame(temp3)

In [63]:
del temp, temp2, temp3

In [124]:
split_documents = df.body.str.split(r'\W+')

In [125]:
docs = []
for doc in split_documents.values.tolist():
    docs.append(set(filter( lambda x: re.match(r'^(\D)+',x),set(map(str.lower, doc)))))
del split_documents

In [126]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [129]:
words = [[w for w in doc if not w in stop_words] for doc in docs]
del docs

In [131]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [[porter.stem(w) for w in word] for word in words]
del words